# T-DAPIA : Two dimensional Analysis Pipeline "Intelligence Artificial"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyimzml.ImzMLParser import ImzMLParser
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import zoom
from bisect import bisect_left, bisect_right
import warnings
import joblib

def generate_tic_map(imzml_file, mzs_range=(600, 1000), sigma=0.5, new_resolution=4.0):
    # Read the imzML file
    p = ImzMLParser(imzml_file)
    spectra = []

    for idx, (x, y, z) in enumerate(p.coordinates):
        mzs, intensities = p.getspectrum(idx)
        # Filter values within the specified mass range
        mask = (mzs >= mzs_range[0]) & (mzs <= mzs_range[1])
        mzs = mzs[mask]
        intensities = intensities[mask]
        spectra.append([mzs, intensities, (x, y, z)])

    x_coords = [x for _, _, (x, _, _) in spectra]
    y_coords = [y for _, _, (_, y, _) in spectra]
    max_x, max_y = max(x_coords), max(y_coords)  # Define these based on your data

    # Create an empty array to store the sum of intensities for each pixel
    sum_intensities = np.zeros((max_y + 1, max_x + 1))

    # Accumulate the sum of intensities for each pixel
    for mzs, intensities, (x, y, z) in spectra:
        sum_intensities[y, x] += np.sum(intensities)

    # Get the indices of non-zero pixels
    real_pixels = np.where(sum_intensities != 0)

    # Crop the sum_intensities array to contain only non-zero pixels
    sum_intensities = sum_intensities[min(real_pixels[0]): max(real_pixels[0]) + 1,
                                      min(real_pixels[1]): max(real_pixels[1]) + 1]

    # Gaussian smoothing
    smoothed_image = gaussian_filter(sum_intensities, sigma=sigma)

    # Increase spatial resolution using interpolation
    zoomed_image = zoom(smoothed_image, new_resolution, order=3)

    # Plot the TIC map
    plt.imshow(zoomed_image, cmap='jet')
    plt.colorbar()
    plt.title("TIC MAP")
    plt.show()


def generate_label_maps(imzml_file, model_file, mass_range=(600, 1000), max_intensity_size=4000, sigma=0.5, new_resolution=4.0, real_pixel_threshold=10000):
    # Suppress warnings
    warnings.filterwarnings("ignore")

    # Load the trained model
    model = joblib.load(model_file)

    # Read the imzML file and preprocess the data
    p = ImzMLParser(imzml_file)
    my_spectra = []  # Initialize my_spectra list
    class_names = model.classes_
    predicted_scores = []

    # Initialize variables for max_x and max_y
    max_x = 0
    max_y = 0

    for idx, (x, y, z) in enumerate(p.coordinates):
        mzs, intensities = p.getspectrum(idx)
        # Filter values within the specified mass range
        mask = (mzs >= mass_range[0]) & (mzs <= mass_range[1])
        mzs = mzs[mask]
        intensities = intensities[mask]
        if len(mzs) == 0:  # Skip spectra with no data in the specified mass range
            continue
        if len(intensities) > max_intensity_size:
            padded_intensities = intensities[:max_intensity_size]
        else:
            padding = max_intensity_size - len(intensities)
            padded_intensities = np.pad(intensities, (0, padding), mode='constant')
        padded_intensities = padded_intensities.reshape(1, -1)  # Reshape to 2D array
        scores = model.predict_proba(padded_intensities)
        predicted_scores.append(scores[0])  # Store the predicted scores for this spectrum
        my_spectra.append((x, y, padded_intensities))  # Collect the spectra data

        # Update max_x and max_y based on coordinates
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    if max_x == 0 or max_y == 0:
        print("No spectra found within the specified mass range.")
        return

    # Create an empty array to store the sum of intensities for each pixel
    sum_intensities = np.zeros((max_y + 1, max_x + 1))

    # Iterate through the spectra and update sum_intensities
    for spectrum in my_spectra:
        x, y, intensities = spectrum
        sum_intensities[y, x] += np.sum(intensities)

    # Get indices of non-zero pixels based on the real_pixel_threshold
    real_pixels = np.where(sum_intensities > real_pixel_threshold)

    # Define colormap
    cmap = 'jet'

    for label_idx, label in enumerate(class_names):
        label_map = np.zeros((max_y + 1, max_x + 1))
        for pixel_idx in range(np.array(real_pixels).shape[1]):
            y, x = real_pixels[0][pixel_idx], real_pixels[1][pixel_idx]
            label_map[y, x] = predicted_scores[pixel_idx][label_idx]  # Use predicted_scores for label map

        # Apply Gaussian smoothing to the label map
        smoothed_label_map = gaussian_filter(label_map, sigma=sigma)

        # Increase the spatial resolution using interpolation
        zoomed_label_map = zoom(smoothed_label_map, new_resolution, order=3)

        plt.figure()
        plt.imshow(zoomed_label_map, cmap=cmap, vmin=0, vmax=1)
        plt.colorbar()
        plt.title(label)

    plt.show()


def calculate_label_ratios(imzml_file, model_file, mass_range=(600, 1000), max_intensity_size=4000, sigma=0, new_resolution=0, real_pixel_threshold=10000):
        # Suppress warnings
    warnings.filterwarnings("ignore")

    # Load the trained model
    model = joblib.load(model_file)

    # Read the imzML file and preprocess the data
    p = ImzMLParser(imzml_file)
    my_spectra = []  # Initialize my_spectra list
    class_names = model.classes_
    predicted_scores = []

    # Initialize variables for max_x and max_y
    max_x = 0
    max_y = 0

    for idx, (x, y, z) in enumerate(p.coordinates):
        mzs, intensities = p.getspectrum(idx)
        # Filter values within the specified mass range
        mask = (mzs >= mass_range[0]) & (mzs <= mass_range[1])
        mzs = mzs[mask]
        intensities = intensities[mask]
        if len(mzs) == 0:  # Skip spectra with no data in the specified mass range
            continue
        if len(intensities) > max_intensity_size:
            padded_intensities = intensities[:max_intensity_size]
        else:
            padding = max_intensity_size - len(intensities)
            padded_intensities = np.pad(intensities, (0, padding), mode='constant')
        padded_intensities = padded_intensities.reshape(1, -1)  # Reshape to 2D array
        scores = model.predict_proba(padded_intensities)
        predicted_scores.append(scores[0])  # Store the predicted scores for this spectrum
        my_spectra.append((x, y, padded_intensities))  # Collect the spectra data

        # Update max_x and max_y based on coordinates
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    if max_x == 0 or max_y == 0:
        print("No spectra found within the specified mass range.")
        return None, None, None  # Return None if no data is found

    # Create an empty array to store the sum of intensities for each pixel
    sum_intensities = np.zeros((max_y + 1, max_x + 1))

    # Iterate through the spectra and update sum_intensities
    for spectrum in my_spectra:
        x, y, intensities = spectrum
        sum_intensities[y, x] += np.sum(intensities)

    # Get indices of non-zero pixels based on the real_pixel_threshold
    real_pixels = np.where(sum_intensities > real_pixel_threshold)
    
    
    
    # Initialize dictionaries to store the sum of scores for each label
    label_sums = {}
    for label in class_names:
        label_sums[label] = 0.0

    # Calculate the sum of scores for each label
    for pixel_idx in range(len(predicted_scores)):
        for label_idx, label in enumerate(class_names):
            label_sums[label] += predicted_scores[pixel_idx][label_idx]

    # Calculate the ratios of each label
    ratios = {}
    for label in class_names:
        ratios[label] = label_sums[label] / np.sum(list(label_sums.values()))

    # Convert ratios dictionary to DataFrame
    df_ratios = pd.DataFrame.from_dict(ratios, orient='index', columns=['Ratio'])

    return df_ratios    
    
def _bisect_spectrum(mzs, mz_value, tol):
    ix_l, ix_u = bisect_left(mzs, mz_value - tol), bisect_right(mzs, mz_value + tol) - 1
    if ix_l == len(mzs):
        return len(mzs), len(mzs)
    if ix_u < 1:
        return 0, 0
    if ix_u == len(mzs):
        ix_u -= 1
    if mzs[ix_l] < (mz_value - tol):
        ix_l += 1
    if mzs[ix_u] > (mz_value + tol):
        ix_u -= 1
    return ix_l, ix_u

def getionimage_and_plot(imzml_file, mz_value, tol=0.5, z=1, reduce_func=sum, sigma=1.0, zoom_factor=1):
    p = ImzMLParser(imzml_file)
    tol = abs(tol)
    im = np.zeros((p.imzmldict["max count of pixels y"], p.imzmldict["max count of pixels x"]))
    
    for i, (x, y, z_) in enumerate(p.coordinates):
        if z_ == 0:
            UserWarning("z coordinate = 0 present, if you're getting blank images set getionimage(.., .., z=0)")
        if z_ == z:
            mzs, ints = map(lambda x: np.asarray(x), p.getspectrum(i))
            min_i, max_i = _bisect_spectrum(mzs, mz_value, tol)
            im[y - 1, x - 1] = reduce_func(ints[min_i:max_i+1])
    
    # Apply Gaussian smoothing
    im_smoothed = gaussian_filter(im, sigma=sigma)
    
    # Zoom the image
    if zoom_factor != 1:
        im_zoomed = zoom(im_smoothed, zoom_factor)
    else:
        im_zoomed = im_smoothed
    
    # Set the title as mz_value
    plt.title(f'mz: {mz_value}')
    
    # Display the image with colorbar
    plt.imshow(im_zoomed, cmap='jet', origin='upper')
    plt.colorbar()
    plt.show()

C:\Users\yanis\AppData\Local\Temp\ipykernel_22768\564639956.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [ ]:
# Example usage for generating a TIC map
imzml_file = "20230703_M15-T1-20um_3.imzML"
generate_tic_map(imzml_file, mzs_range=(600, 1000), sigma=0.5, new_resolution=4.0)

In [ ]:
# Example usage for generating label maps
model_file = "Model_mices_cell_lines_Combined25092023.pkl"
generate_label_maps(imzml_file, model_file, mass_range=(600, 1000), max_intensity_size=4000, sigma=0.0, new_resolution=4.0, real_pixel_threshold=20000)

In [ ]:
# Example usage for generating scores
df_ratios = calculate_label_ratios(imzml_file, model_file)
df_ratios

In [ ]:
# Example usage for generating an ion image and plotting it
mz_value = 736.55
getionimage_and_plot(imzml_file, mz_value, tol=0.5, z=1, reduce_func=sum, sigma=0.1, zoom_factor=4)